FaceValue Bioinstrumentation Project | Coded by Cata Santos Anzola | Finished 26/11/2025

cata comment: please run the arduino code before running this code! This code will receive the serial output from the arduino (stored in the SD card), and put it through our EMG-classifying model. The outcome of the model will then print out a string that will be sent to the arduino to show on the LCD.

Step 0. Import libraries

In [ ]:
import numpy as np
import pandas as pd
from Arduino import Arduino
import time
import pickle
import joblib
from keras.models import load_model
import serial
import sys
import time
import os
import matplotlib
# cata comment: Import necessary libraries, do not change tthisssss

Step 1. Connect with Arduino

In [ ]:
# cata comment: begin arduino connection -> will be bypassing use of actual arduino coding system through python's arduino library
number = 3; #"number" is the arduino port for the pin connection, change this here if you need to
board = Arduino(number, port="COM3") # "board" will now be the name for our arduino, it is specified through its COM and number

Step 2. Receive SD Card Data

In [ ]:
# cata comment: this part of the code will run after getting the Arduino to ouput the Serial.monitor data into the SD card.
# We will read the data from the SD card, process it, and then input it into the model to get the prediction.

# remember that if you change the directory, you have to change it on the "path" for this to work.
# cata comment: remember to copy this per each input (1-5)



path = "C:/Users/catas\OneDrive - Clemson University/Uni/Bioinstrumentation Lab/FaceValue/ArduinoEMGCode" #establish directory of SD card data (SD card filepath)
data = pd.read_json(path) #read the .json file from SD card (file should be the serial monitor output)
plt.plot(data) #plot the data for visualization
plt.show()
data = np.array(data).reshape(1, -1)  # reshape for model input, do not change this or it won't work.

Step 3. Run EMG signal into ML model for classification

In [ ]:
#with our SD card data now formatted, we can load the model and get the prediction:

model = load_model('C:/Users/catas/OneDrive - Clemson University/Uni/Bioinstrumentation Lab/FaceValue/EMGClassifier.hdf5') # load in the .hd5 model that was trained from ALS/control datasets
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy']) #compile the model to have an outcome based on accuracy and loss
outcome=model.predict(data) #get the prediction from the model based on the data from SD card
print(outcome) #print this so we can see it as we run & trouble shoot, comment out later if needed


#cata comment: this inequality isn't final as we need to see how each EMG weighs into the prediction.
if outcome != 0:
    stringa = 'ALS Suspected'
else:
    stringa = 'ALS Not Suspected'

Step 4. output string from model result to arduino

In [ ]:
board.write(bytes(stringa, 'utf-8'))
board.close()

           ,....,
        ,;;:o;;;o;;,
      ,;;o;'''''';;;;,
     ,;:;;        ;;o;,
     ;o;;          ;;;;        Happy Holidays <3
     ';;;,  _  _  ,;;;'
      ';o;;/_\/_\;;o;'
         ';;\_\/_/;;'
           '//\\'
           //  \\
          |/    \|